
## Shor's algorithm

* break RSA


In [1]:

from math import gcd,log
from random import randint

import numpy as np

import qiskit

from qiskit import *
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister, register
from qiskit import *
from qiskit import IBMQ
from qiskit import BasicAer


In [2]:

qasm_sim = qiskit.Aer.get_backend('qasm_simulator')


In [3]:

def period(a, N):
    
    available_qubits = 16
    r = -1

    if N >= 2 ** available_qubits:
        print(str(N) + ' is too big for IBMQX')

    qr = QuantumRegister(available_qubits)
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr, cr)
    x0 = randint(1, N - 1)
    x_binary = np.zeros(available_qubits, dtype=bool)

    for i in range(1, available_qubits + 1):
        bit_state = (N % (2 ** i) != 0)
        if bit_state:
            N -= 2 ** (i - 1)
        x_binary[available_qubits - i] = bit_state

    for i in range(0, available_qubits):
        if x_binary[available_qubits - i - 1]:
            qc.x(qr[i])
    x = x0

    while np.logical_or(x != x0, r <= 0):
        r += 1
        qc.measure(qr, cr)
        for i in range(0, 3):
            qc.x(qr[i])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[2])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[3], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])

        result = execute(qc, backend=qasm_sim, shots=1024).result()
        counts = result.get_counts()
        print(qc)
        results = [[], []]
        for key, value in counts.items():  # the result should be deterministic but there might be some quantum calculation error so we take the most reccurent output
            results[0].append(key)
            results[1].append(int(value))
        s = results[0][np.argmax(np.array(results[1]))]
    return r



In [4]:

def shors_breaker(N):
    N = int(N)
    while True:
        a=randint(0,N-1)
        g=gcd(a,N)
        if g!=1 or N==1:
            return g,N//g
        else:
            r=period(a,N)
            if r % 2 != 0:
                continue
            elif pow(a,r//2,N)==-1:
                continue
            else:
                p=gcd(pow(a,r//2)+1,N)
                q=gcd(pow(a,r//2)-1,N)
                if p==N or q==N:
                    continue
                return p,q



## Number to factorize 

* 3233 = 61 x 53


In [5]:

N = 3233


In [6]:

N


3233

In [7]:

## assert N>0,"Input must be positive"



## Run Quantum algorithm


In [8]:


res = shors_breaker(N)



       ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q0_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
       └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q0_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
            └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q0_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
             ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q0_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
             ║   ║   └╥┘  ┌─┐                        ║                     »
 q0_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
       ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q0_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
       └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘                »

        ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q21_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
        └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q21_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
             └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q21_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
              ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q21_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
              ║   ║   └╥┘  ┌─┐                        ║                     »
 q21_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
        ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q21_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
        └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘    

        ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q42_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
        └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q42_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
             └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q42_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
              ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q42_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
              ║   ║   └╥┘  ┌─┐                        ║                     »
 q42_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
        ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q42_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
        └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘    

        ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q63_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
        └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q63_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
             └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q63_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
              ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q63_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
              ║   ║   └╥┘  ┌─┐                        ║                     »
 q63_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
        ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q63_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
        └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘    

        ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q77_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
        └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q77_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
             └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q77_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
              ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q77_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
              ║   ║   └╥┘  ┌─┐                        ║                     »
 q77_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
        ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q77_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
        └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘    

        ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q98_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
        └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q98_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
             └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q98_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
              ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q98_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
              ║   ║   └╥┘  ┌─┐                        ║                     »
 q98_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
        ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q98_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
        └───┘ ║   ║    ║    ║   ┌─┐                   ║  └╥┘    

         ┌───┐                                        ┌─┐┌───┐┌───┐     ┌───┐»
 q119_0: ┤ X ├────────────────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├»
         └───┘┌─┐┌───┐     ┌───┐     ┌───┐            └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘»
 q119_1: ─────┤M├┤ X ├─────┤ X ├──■──┤ X ├─────────────╫────────■──┤ X ├──■──»
              └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘             ║           └───┘     »
 q119_2: ──────╫──┤M├─┤ X ├──■──┤ X ├──■───────────────╫─────────────────────»
               ║  └╥┘ └┬─┬┘     └───┘                  ║                     »
 q119_3: ──────╫───╫───┤M├─────────────────────────────╫─────────────────────»
               ║   ║   └╥┘  ┌─┐                        ║                     »
 q119_4: ──────╫───╫────╫───┤M├────────────────────────╫─────────────────────»
         ┌───┐ ║   ║    ║   └╥┘                        ║  ┌─┐                »
 q119_5: ┤ X ├─╫───╫────╫────╫─────────────────────────╫──┤M├────────────────»
         └───┘ ║   ║    ║    ║   ┌─┐                

In [9]:

print( res )


(53, 61)


In [10]:

61 * 53 


3233